## INTRODUCTION

Good afternoon.  In my free time I enjoy experimenting with AI.  Not just solving a problem but playing around with models and parameters to get a feel for what's really going on.  One of the topics I find most interesting is reinforcement learning (RL).  Instead of fitting a model to a dataset, RL forces us to be creative in creating an agent that experiences the data in the form of the game state and labels in the form of rewards.  Therefore, not only do we have to be careful in how we design the model, but we need to consider how we construct the game, inputs, and reward system.  However, despite the freedom inherent in reinforcement learning, there exists an underlying structure to each problem that the model is trying to learn.  Hopefully, during the course of this notebook I'll successfully illustrate both the structure behind RL as well as the methodology behind reinforcement learning as a whole.

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/sample_random.gif?raw=true)

## Cartpole

In order to best discuss the core concepts behind RL, I've decided to use the simplest use case known as the cartpole problem.  It consists of a pole standing on top of a cart.  The player can accelerate the cart right or left, moving the pivot point in an attempt to prevent the pole from falling.  The model is fed four different inputs: the x position of the cart ($x$), the x velocity of the cart ($\dot x$), the angular position of the pole ($\theta$), and the angular velocity of the pole ($\dot \theta$).

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/ObservationSpace.PNG?raw=true)

At the beginning of the game, each of the four values are randomly initialized between -0.05 and 0.05 so that each initial state is slightly different but close to zero.  The game ends when either the absolute value of the x position exceeds 2.4, the pole angle exceeds 41.8$^{\circ}$, or the game lasts for 500 frames.  The limitation on the cart is a lot more lenient than the pole, which means any agent training to balance the pole will prioritize the pole's position over the cart's.  The agent has two options; accelerate the cart left or right.

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/ActionSpace.PNG?raw=true)

Note that the agent is not allowed to keep the current speed.  The agent is also not allowed to directly adjust the pole speed.  Instead it must move the cart and use the resultant torque to keep the pole standing up.

## Physics

Before I get into the models I've used, let's talk about the environment and the physics behind it.  The cartpole is an example of an equilibrium problem, where we are trying to maintain a certain position (pole being straight up).  Furthermore, this is a type of equilibrium that is known as unstable equilibrium where the slightest deviation from the target position leads to further deviation without intervention.  As the horizonal distance limit is much larger than the angle limit, we can ignore it right now in order to simplify our analysis of the environment.  The two relevant equations are 

$$\dfrac{\delta \theta}{\delta t} \approx \dot \theta$$
$(1)$

$$\dfrac{\delta \dot \theta}{\delta t} \approx \sin \theta$$
$(2)$

Anyone who has studied physics will know that $\sin \theta \approx \theta$ for small angles as this is a very popular trick in derivations.  Since the purpose of the cartpole to is maintain small angular displacement from standing straight up, we can safely assume angles will be "small" enough.  Therefore, it seems to me that we can solve an equation balancing pole with a linear model as both $\theta$ and $\dot \theta$ rely on the other in a linear relationship.  Keep in mind that we are briefly ignoring the x constraint.  To further describe the equilibrium system in this way, here are two graphs.


![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/PotentialEnergyGraph.png?raw=true)|![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/PhaseSpaceDiagram.png?raw=true)
-|-
(A)|(B)

Figure A demonstrates the instability of the potential energy.  This is a good initial assessment as every system tends to move towards lowest potential energy.  Figure B is known as a phase space diagram, where we graph the rate of change in terms of both position and velocity.  As we can see, the further the angle is from zero, the greater the acceleration (rate of change of velocity).  The greater the angular velocity, the greater the rate of change of angle (obviously, but what matters it that the rate of change is pointed away from the center or equilibrium point).  While describing the same phenomena, the phase space diagram does a much better job because we can see that the velocity is much more dangerous to equilibrium than the angle (this is because the game ends at smaller angles so gravity does not get a chance to really affect the pole).  Another thing to note is that the velocity values will seem higher than possible.  This is because the game runs at 50 fps so that any model has the ability to update the speed every 0.02 seconds.  The dynamics of the system can be explored with the Lagrangian and the Euler-Lagrange equation.

$$\begin{equation}L = T - V\end{equation}$$
$(3)$


$$\dfrac{d}{dt}\dfrac{\delta L}{\delta\dot q} = \dfrac{\delta L}{\delta q}$$
$(4)$

Eq (3) sets the lagrangian (L) equal to the total kinetic energy of the system (T) minus the total potential energy of the system (V).  As there is no friction, the lagrangian will be a constant.  Deriving lagrangian mechanics is a little out of the scope for this report, but I'll leave links at the end for those interested.  The first link is a mathematical proof that the Euler-Lagrange equation solves for the shortest path between two points and the second provides an explanation that eq (4) solves for the shortest path through the energy of the system.  For now, I'll keep with the assumption that solving eq (4) properly will solve the practical dynamics of the cartpole (and any similar RL environments).  The EL equation states that the derivative of the lagrange with respect to the spatial coordinates (q) must be equal to the derivative with respect to time of the derivative with respect to the rate of change of the spatial coordinates of the lagrange.

## Baselines

Before we build a model, the first step is to establish simple heuristics in order to ground our expectations.  I played the cartpole myself and got an average around 40.  I could have practised and gotten better at the game but I'd like to avoid cartpole tunnel syndrome.  The Suicide heuristic is one that attempts to accelerate the pole towards the ground, ending on an average of 9 frames.  This is the worst a model can perform and is what happens when a model only predicts one direction.

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/suicide_model_sample.gif?raw=true)



Playing randomly averages around 22 frames.  This is the best baseline, as any model that outperforms this will have at least learned something about the environment.  While playing randomly extends the duration of the game, we can see in the phase space diagram that the heuristic is incapable of reacting to the environment, allowing the instability to increase and push it away from the center.

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/random_model_sample.gif?raw=true)


Controlling the angle, or accelerating to the right when the angle is positive and to the left when the angle is negative, yields an average of 41 frames.  This heuristic is able to adapt to the environment, but quickly gains energy and momentum that spirals the pole away from the center.


![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/controlled_angle_sample.gif?raw=true)

Because the angular velocity is more volatile than the angle itself, it's no surprise that controlling the velocity yields an average of 200 frames.  On an initial glance, it may seem that the heuristic works perfectly well and that the only issue is the $x$ constraint.  However, the phase space diagram exposes the fact that the angle is slowly increasing as well.  This is because the greater the angle, the more the pole tends to increase speed in that direction.  Merely controlling the speed will then cause the pole to slip as the average speed will be slightly in favor of increasing towards the pole's fall.

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/controlled_velocity_sample.gif?raw=true)

## Reinforcement Learning

Now, before we can begin building models, I'd like to define a few terms and explain the concept behind reinforcement learning.  Reinforcement learning (RL) has three main components:

1. <b>Environment</b>.  The environment is the ruleset, underlying physics, reward system, and anything else that is not the agent.  
2. <b>Actions</b>.  Actions are a list of possible behaviours that the agent can take.  While the list of available actions are determined by the state
3. <b>Noniterable Dataloader</b>.  Another issue was that I generated image augmentation during dataset creation, outside of the dataloader.  Even though each image was turned into 15 images, they were the same 15 images regardless of epoch.  This is a somewhat minor detail, but I made adjustments to the ImageDataset class object that performed the rotation and cropping of the image with each iteration. Therefore, each epoch of the original is equal to ~15 epochs of the new dataset.  However, I was able to train for 400 epochs for the new model and only ran for 10 epochs on the old.
4. <b>Image Sizes</b>.  Another big issue has to do with the size of the image.  Both models perform better when given an image the same dimensions as the data they were trained on.  In theory, I could have randomly resized the training data in order to compensate for this weakness, but this did not occur to me until I started writing this report and comparing the two models.  Regardless, another big impact that size had was the capacity of the GPU.  I had to make cuts to the neural networks and only run on a batch size of 1 in order to be able to fit the 600X800 images.  However, when I converted all images to 100X100 I was not only able to maintain the integrity of the UNet structure and run batches of 16, but I was able to train both the TNet and MNet together.  I named this new network the GluNet (GNet).  This is a much better strategy as it allows for both networks to learn from each other and work together.

## Markov



In [5]:
import numpy as np
import gym

class MarkovAgent():
    
    def __init__(self):
        
        self.memory = []
        self.Qs = {}
        self.Ns = {}
        self.done = False
        self.gamma = 0.9
        self.epsilon = 1
        self.decay = 0.99
        
        self.session_score = 0
        self.max_distance = 0
        
        self.env = gym.make("CartPole-v1")
        
    def reset(self):
        
        self.done = False
        self.memory = []
        
    def play_session(self):
        
        observation = self.env.reset()
        
        self.session_score = 0
        self.max_distance = 0
        
        while not self.done:
            
            s,action = self.take_action(observation)
            observation, reward, self.done, info = self.env.step(action)
            self.session_score += 1            
            self.memory.append([s,-100 if self.done and self.session_score<500 else reward,action])
            
            distance = observation[0]
            
            if abs(distance)>self.max_distance:
                self.max_distance = abs(distance)
            
        past_reward = 0
        
        for s, reward, action in reversed(self.memory):
            
            self.Qs[s][action] = (self.Qs[s][action]*self.Ns[s][action] + (reward + self.gamma*past_reward))/(self.Ns[s][action]+1)
            self.Ns[s][action] += 1
            past_reward = (reward + self.gamma*past_reward)
            
        self.epsilon = max(self.decay*self.epsilon,0.05)
        self.reset()
        
        
    def take_action(self,observation):
        
        s = np.asarray([round(observation[i],1+int(i/2)) for i in range(len(observation))]).tostring()
        if s not in self.Ns:
            
            self.Ns[s] = [5,5]
            self.Qs[s] = [100,100]
            
        q_list = self.Qs[s]
        
        if np.random.uniform(0,1)>self.epsilon:
            
            action = np.argmax(q_list)
            
        else:
            
            action = self.env.action_space.sample()
            
        return s,action

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/training_markov.gif?raw=true)

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/markov_decisionboundary.png?raw=true)|![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/svm_decisionboundary.png?raw=true)
-|-
A|B


I increased limit to 1000 just to test the model and wowza


![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/svm_model_sample.gif?raw=true)

## Linear


In [1]:
import numpy as np
import gym
from collections import deque


class LinearModel():
    
    def __init__(self,input_dim,output_dim):
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.matrix = np.random.rand(self.input_dim,self.output_dim)
        self.bias = np.random.uniform(-1,1)
        
    def predict(self,observation):
        
        return np.sum([self.matrix[i]*observation[i] for i in range(self.input_dim)],axis=0) + self.bias
    
    def update(self,instance,lr):
        
        state = instance[:4]
        action = instance[4]
        reward = instance[5]
        
        forward_feed = [self.matrix[i][action]*state[i] for i in range(self.input_dim)]
        error = reward - (np.sum(forward_feed) + self.bias)
        adjustment = error*lr
        
        self.bias += adjustment
        self.matrix[:,action] += np.asarray(state)*adjustment



class CartpoleAgent():
    
    def __init__(self,minmem,maxmem):
        
        self.minmem = minmem
        self.maxmem = maxmem
        self.memory = deque([], maxlen=self.maxmem)
        self.epsilon = 1
        self.decay = 0.995
        self.gamma = 0.95
        self.lr = 0.001
        
        self.env = gym.make("CartPole-v1")
        self.model = LinearModel(4,2)

        self.scores = []
        self.means = []
        self.losses = []

    def predict(self,observation):
        
        return self.model.predict(observation)
    
    def remember(self,batch_size):
        
        return random.sample(self.memory,batch_size)
    
    def train(self,epochs,num_games,batch_size):
        
        e = 0
        
        while e<epochs:
            
            for _ in range(num_games):
            
                done = False
                observation = self.env.reset()
                trainexamples = []
                session_score = 0

                while not done:

                    if np.random.uniform(0,1)>self.epsilon:
                        action = np.argmax(self.model.predict(observation))
                    else:                    
                        action = self.env.action_space.sample()

                    session_score += 1
                    next_state, reward, done, info = self.env.step(action)
                    trainexamples.append([observation,action,-100 if done and session_score<500 else 1,next_state])
                    observation = next_state

                self.scores.append(session_score)

                current_reward = 0
                for example in reversed(trainexamples):
                    current_reward = example[2] + self.gamma*current_reward
                    self.memory.append([example[0][0],example[0][1],example[0][2],example[0][3],example[1],current_reward])                 
                     
                    
            mean = np.mean(self.scores[-num_games:])
            print(f'Epoch {e+1} had a mean score of {mean} over {num_games} games.')
            self.means.append(mean)
            
                
            
            self.epsilon = max(0.05,self.decay*self.epsilon)
            
            if len(self.memory)>self.minmem:
                
                batch = self.remember(batch_size)
                for b in batch:
                    self.model.update(b,self.lr)
                    
                e += 1

![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/LinearModelTrainingwoAgency.png?raw=true)|![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/LinearModelTrainingwAgency.png?raw=true)
-|-
![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/linear_model_random.gif?raw=true)|![Original Image](https://github.com/alexikerd/Cartpole/blob/main/graphics/linear_model_best.gif?raw=true)

## Citations


1. Derivation of Euler-Lagrange equation https://farside.ph.utexas.edu/teaching/336L/Fluid/node266.html
2. Explanation of lagrangian mechanics http://www.physicsinsights.org/lagrange_1.html
3. Solution to Cartpole problem using physics https://danielpiedrahita.wordpress.com/portfolio/cart-pole-control/
4. How to handle P(s) and Q(s) with state as input https://web.stanford.edu/~surag/posts/alphazero.html
5. Good analysis of simple models to solve cartpole http://kvfrans.com/simple-algoritms-for-solving-cartpole/
6. Stackoverflow covering linear model to solve cartpole https://stats.stackexchange.com/questions/250531/understanding-oscillating-behaviour-when-using-q-learning-on-cart-pole-problem
7. Discussion on overfitting in Deep RL https://arxiv.org/pdf/1804.06893.pdf